In [1]:
import requests
import json
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

### 위도/경도 결측치 보간 by addr

In [22]:
def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
    headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']), float(match_first['x'])

getLatLng('서울 마포구 모래내로1길 20')

(37.5656948713242, 126.903284318892)

In [45]:
url = 'https://dapi.kakao.com/v2/local/search/address.json?query=서울 마포구 모래내로1길 20'
headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} 
result = json.loads(str(requests.get(url, headers=headers).text))

result['documents'][0]['address']

{'address_name': '서울 마포구 성산동 590-4',
 'b_code': '1144012500',
 'h_code': '1144073000',
 'main_address_no': '590',
 'mountain_yn': 'N',
 'region_1depth_name': '서울',
 'region_2depth_name': '마포구',
 'region_3depth_h_name': '성산2동',
 'region_3depth_name': '성산동',
 'sub_address_no': '4',
 'x': '126.903284318892',
 'y': '37.5656948713242'}

In [47]:
charger = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/api_charger_final.txt')
print(charger.shape)
charger.head()

(3559, 8)


,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동,1168064000
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동,1168052100
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동,1168064000
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동,1168064000
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동,1168052100


In [54]:
na = charger[charger['lat']==33] 
na.head()

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
802,서울특별시 광진구 능동로 19,33.0,124.0,자양동,4,1121510500,자양4동,1121584700
895,서울특별시 구로구 경인로8길 64,33.0,124.0,오류동,10,1153010800,오류2동,1153078000
967,서울특별시 구로구 디지털로34길 27,33.0,124.0,구로동,2,1153010200,구로3동,1153054000
1201,서울특별시 노원구 동일로 1127,33.0,124.0,공릉동,3,1135010300,공릉1동,1135059500
1321,서울특별시 도봉구 도봉로136가길 73,33.0,124.0,창동,8,1132010700,창5동,1132051500


In [57]:
for i in na.index:
    charger['lat'][i] = getLatLng(charger['addr'][i])[0]
    charger['lng'][i] = getLatLng(charger['addr'][i])[1]
    
charger[charger['lat']==33] 

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드


In [58]:
charger.iloc[na.index]

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
802,서울특별시 광진구 능동로 19,37.533153,127.067236,자양동,4,1121510500,자양4동,1121584700
895,서울특별시 구로구 경인로8길 64,37.492168,126.840292,오류동,10,1153010800,오류2동,1153078000
967,서울특별시 구로구 디지털로34길 27,37.485473,126.898223,구로동,2,1153010200,구로3동,1153054000
1201,서울특별시 노원구 동일로 1127,37.629590,127.069855,공릉동,3,1135010300,공릉1동,1135059500
1321,서울특별시 도봉구 도봉로136가길 73,37.655653,127.042805,창동,8,1132010700,창5동,1132051500
1325,서울특별시 도봉구 도봉로169길 31,37.679952,127.044238,도봉동,3,1132010800,도봉1동,1132052100
1392,서울특별시 도봉구 해등로16길 12,37.656235,127.040573,창동,5,1132010700,창5동,1132051500
1413,서울특별시 동대문구 망우로 95,37.591185,127.063963,휘경동,8,1123010900,휘경1동,1123072000
1508,서울특별시 동대문구 황물로15길 25,37.566563,127.057188,답십리동,6,1123010500,답십리2동,1123061000
1592,서울특별시 동작구 서달로 91,37.503166,126.958891,흑석동,10,1159010500,흑석동,1159060500


In [59]:
print(charger.shape)
charger.head()

(3559, 8)


,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동,1168064000
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동,1168052100
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동,1168064000
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동,1168064000
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동,1168052100


### 위도/경도 결측치 보간 완료!

In [60]:
charger.to_csv('/Users/kimmh/Desktop/서데캠/data/api_charger_jjinfinal.txt', sep=',', index=False)

---

---

---

In [2]:
pop = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/TB_T_RSPOP_ADMI_2107.txt')
print(pop.shape)
pop.head()

(17766, 5)


,STD_YM,ADMI_CD,ADMI_NM,RSPOP_CNT,CTY_NM
0,202107,11200540,마장동,265,성동구
1,202107,11290555,삼선동,205,성북구
2,202107,11545510,가산동,177,금천구
3,202107,11560605,문래동,769,영등포구
4,202107,11470560,신월1동,215,양천구


In [11]:
won = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/TB_T_RSPOP_ADMI.txt', sep='|')
print(won.shape)
won.head()

(2046404, 7)


,STD_YM`STD_YM`,`ADMI_CD`,`ADMI_NM`,`SEXDSTN_CD`,`AGRDE_CD`,`RSPOP_CNT`,`CTY_NM`
0,`201807`,`11230660`,`장안2동`,`1`,`0004`,`667`,`동대문구`
1,`201807`,`11590660`,`대방동`,`1`,`0004`,`599`,`동작구`
2,`201807`,`11170700`,`보광동`,`1`,`0004`,`156`,`용산구`
3,`201807`,`11290525`,`성북동`,`1`,`0004`,`210`,`성북구`
4,`201807`,`11620585`,`낙성대동`,`1`,`0004`,`228`,`관악구`


In [21]:
won[won['STD_YM`STD_YM`']=='`202107`']['`ADMI_NM`'].nunique()

422

In [7]:
pop['ADMI_CD'].nunique()

423

In [4]:
dt = pop.groupby(['ADMI_NM', 'ADMI_CD'])
gr = dt.sum('RSPOP_CNT')

gr.reset_index(drop=False, inplace=True)

gr.head()

,ADMI_NM,ADMI_CD,STD_YM,RSPOP_CNT
0,가락1동,11710631,8488494,27473
1,가락2동,11710632,8488494,32575
2,가락본동,11710620,8488494,26564
3,가리봉동,11530595,8488494,8954
4,가산동,11545510,8488494,23377


In [5]:
print(gr.shape)

(423, 4)


In [6]:
sum(gr['RSPOP_CNT'])

9493265

In [39]:
gr.columns = ['행정동', '행정동코드', '기준년월', '인구']

gr2 = gr[['행정동코드', '행정동']]
gr2['행정동코드'] = gr2['행정동코드']*100

print(gr2.shape)
gr2.head()

(423, 2)


,행정동코드,행정동
0,1171063100,가락1동
1,1171063200,가락2동
2,1171062000,가락본동
3,1153059500,가리봉동
4,1154551000,가산동


In [40]:
hang = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/행정동코드.txt')
print(hang.shape)
hang.head()

(426, 2)


,행정동코드,행정동
0,1111051500,청운효자동
1,1111053000,사직동
2,1111054000,삼청동
3,1111055000,부암동
4,1111056000,평창동


### hang: 행정동코드 / gr2: 동별거주인구(2107) >> 3개 차이

In [41]:
dup = pd.concat([hang, gr2])
print(dup.shape)
dup.head()

(849, 2)


,행정동코드,행정동
0,1111051500,청운효자동
1,1111053000,사직동
2,1111054000,삼청동
3,1111055000,부암동
4,1111056000,평창동


In [44]:
dupx = dup.drop_duplicates(['행정동코드'], keep=False)
dupx

,행정동코드,행정동
275,1153080000,항동
408,1174052500,상일1동
409,1174052600,상일2동


### 항동 / 상일1동 / 상일2동
### 위 3개가 동별 거주인구 데이터와 행정동 코드 데이터의 차이  (거주인구 데이터에 없는 것)